In [6]:
import pandas as pd
import numpy as np
import re

In [ ]:
#Criando arquivo com as disciplinas
dados = pd.read_csv('../dados/oferta_2025_1_depto-estatística---brasília_graduação.csv')
dados['id_disciplina'] = dados.apply(lambda x: x['codigo_disciplina'] + '_' + str(x['codigo_turma']).zfill(2), axis =1)
dados[['id_disciplina', 'disciplina', 'carga_horaria']].to_csv('../dados/disciplinas.csv', index=False)

In [ ]:
#Criando matriz de conflitos
def parse_horario(horario_str):
    """
    Interpreta uma string de horário no formato '24M12' e a converte
    em um dicionário estruturado.

    Args:
        horario_str (str): A string de horário (ex: '35T34', '6N12', etc.).

    Returns:
        dict: Um dicionário com os dias, turno e horários, ou None se o formato for inválido.
    """
    # Expressão regular para capturar os 3 componentes: dias, turno, blocos
    match = re.match(r'(\d+)([MTN])(\d+)', horario_str)
    
    if not match:
        return None # Formato inválido

    dias_str, turno_char, blocos_str = match.groups()

    mapa_dias = {'2': 'SEG', '3': 'TER', '4': 'QUA', '5': 'QUI', '6': 'SEX'}
    
    # Converte os dígitos dos dias em um conjunto de strings (ex: {'SEG', 'QUA'})
    dias_set = {mapa_dias[dia] for dia in dias_str if dia in mapa_dias}
    
    # Converte os dígitos dos blocos em um conjunto de inteiros (ex: {1, 2})
    blocos_set = {int(bloco) for bloco in blocos_str}

    return {
        'dias': dias_set,
        'turno': turno_char,
        'blocos': blocos_set
    }

def verificar_conflito(horario_str1, horario_str2):
    """
    Verifica se duas strings de horário entram em conflito.

    Args:
        horario_str1 (str): Horário da primeira disciplina.
        horario_str2 (str): Horário da segunda disciplina.

    Returns:
        bool: True se houver conflito, False caso contrário.
    """
    h1 = parse_horario(horario_str1)
    h2 = parse_horario(horario_str2)

    if h1 is None or h2 is None:
        print(f"Aviso: Formato de horário inválido encontrado ({horario_str1} ou {horario_str2})")
        return False

    # 1. Verifica se os turnos são diferentes (se forem, não há conflito)
    if h1['turno'] != h2['turno']:
        return False

    # 2. Verifica se há intersecção de dias da semana
    dias_em_comum = h1['dias'].intersection(h2['dias'])
    if not dias_em_comum:
        return False

    # 3. Verifica se há intersecção de blocos de horário
    blocos_em_comum = h1['blocos'].intersection(h2['blocos'])
    if not blocos_em_comum:
        return False

    # Se passou por todas as verificações, significa que há conflito
    return True


disciplinas = dados['id_disciplina'].tolist()
horarios = dados.set_index('id_disciplina')['horario'].to_dict()
num_disciplinas = len(disciplinas)
matriz_conflitos = pd.DataFrame(0, index=disciplinas, columns=disciplinas)


for i in range(num_disciplinas):
    for j in range(i, num_disciplinas):
        disc1 = disciplinas[i]
        disc2 = disciplinas[j]
        
        # Não compara uma disciplina com ela mesma
        if i == j:
            continue
            
        if verificar_conflito(horarios[disc1], horarios[disc2]):
            matriz_conflitos.loc[disc1, disc2] = 1
            matriz_conflitos.loc[disc2, disc1] = 1
matriz_conflitos.to_csv('../dados/matriz_conflitos_disciplinas.csv', index=True)

In [ ]:
#Criando arquivo com os professores
prof1 = pd.read_csv('../dados/oferta_2025_1_depto-estatística---brasília_graduação.csv')
prof2 = pd.read_csv('../dados/oferta_2025_2_depto-estatística---brasília_graduação.csv')
prof = pd.concat([prof1, prof2])
prof = prof.groupby(['docente', 'ano_periodo']).sum()[['carga_horaria']].reset_index().groupby(['docente']).mean()[['carga_horaria']].reset_index()
prof['id_docente'] = prof.reset_index().apply(lambda x: 'docente' + '_' + str(x['index'] +1).zfill(2), axis=1)
prof[['id_docente', 'docente', 'carga_horaria']].to_csv('../dados/docentes.csv', index=False)

In [94]:
#Simulando as preferências
PESOS_PREFERENCIAS = [0.65, 0.15, 0.15, 0.05]
prof_ids_pref = []
disc_ids_pref = []
preferencias = []

# Loop para gerar a preferência de cada professor por cada disciplina
for prof_id in prof['id_docente']:
    for disc_id in dados['id_disciplina']:
        prof_ids_pref.append(prof_id)
        disc_ids_pref.append(disc_id)
        # Gera a preferência usando os pesos definidos
        pref = np.random.choice([0, 1, 2, 3], p=PESOS_PREFERENCIAS)
        preferencias.append(pref)

df_preferencias = pd.DataFrame({
    'id_docente': prof_ids_pref,
    'id_disciplina': disc_ids_pref,
    'preferencia': preferencias
})

df_preferencias.to_csv('../dados/preferencias.csv', index=False)